In [1]:
import pandas as pd
import numpy as np

import sqlite3
import requests

from db_wrapper import request_db

db = request_db('database')

In [2]:
cleared_lines = []
with open('logs.txt') as f:
    for line in f.readlines():
        cleared_line = " ".join(line.replace('|','').split()).split()
        cleared_line[6] = cleared_line[6].replace('https://all_to_the_bottom.com/', '')
        if cleared_line[6] == '':
            cleared_line[6] = 'index'
        cleared_line[1] = cleared_line[1] + ' ' + cleared_line[2]
        cleared_lines.append(cleared_line)

df = pd.DataFrame(cleared_lines, columns=[0, 'date', 'time', 'id', 4, 'ip', 'action']).drop([0, 4], axis=1)
df = df.replace('', np.nan).dropna()
df.reset_index(drop=True, inplace=True)

In [3]:
conn = sqlite3.connect("database")
cursor = conn.cursor()

cursor.execute("""DROP TABLE IF EXISTS Users""")
cursor.execute("""DROP TABLE IF EXISTS Actions""")
cursor.execute("""DROP TABLE IF EXISTS Transasction""")
cursor.execute("""DROP TABLE IF EXISTS Products""")

cursor.execute("""CREATE TABLE IF NOT EXISTS Users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
                    ip TEXT NOT NULL UNIQUE,
                    country TEXT
                    );
               """)
cursor.execute("""CREATE TABLE IF NOT EXISTS Actions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
                    path TEXT NOT NULL,
                    dtime DATETIME,
                    u_id INTEGER REFERENCES Users (id)
                    );
               """)
cursor.execute("""CREATE TABLE IF NOT EXISTS Transactions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
                    finished BOOL DEFAULT 'False',
                    dtime DATETIME,
                    u_id INTEGER REFERENCES Users (id)
                    );
               """)
cursor.execute("""CREATE TABLE IF NOT EXISTS Products (
                    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
                    category TEXT NOT NULL,
                    name TEXTs NOT NULL
                    );
               """)

data = cursor.fetchall()
conn.close()

In [4]:
for i, ip in enumerate(df.ip.unique()):
    try:
        response = requests.get(f'http://ipinfo.io/{ip}?token=75c7c09e5f27cd')
        country = response.json()['country']
    except:
        country = 'Undefiend'
    print(i, ip, country, sep='\t')
    db.request_insert_two('Users', 'ip, country', ip, country)

	US
104	43.126.202.130	JP
105	106.32.135.104	CN
106	238.201.179.200	Undefiend
107	199.113.38.34	US
108	147.42.83.4	US
109	148.110.179.251	LU
110	244.112.141.107	Undefiend
111	54.142.244.244	US
112	44.171.202.255	US
113	170.42.135.135	US
114	172.27.107.17	Undefiend
115	237.134.190.255	Undefiend
116	247.143.216.108	Undefiend
117	190.36.225.207	VE
118	222.133.115.74	CN
119	117.200.172.59	IN
120	160.227.79.231	US
121	75.253.63.156	US
122	250.245.205.101	Undefiend
123	230.0.255.166	Undefiend
124	151.79.112.77	IT
125	222.45.208.201	CN
126	197.161.230.145	EG
127	117.48.37.148	CN
128	77.234.234.235	SK
129	90.193.32.160	GB
130	5.68.141.137	GB
131	111.10.43.60	CN
132	25.226.9.148	GB
133	160.54.56.36	DE
134	46.103.22.133	GR
135	185.173.181.152	LU
136	40.231.92.112	US
137	173.56.117.58	US
138	243.200.44.96	Undefiend
139	99.60.82.81	US
140	110.241.111.50	CN
141	165.176.232.246	US
142	219.14.98.172	JP
143	8.165.58.237	CN
144	254.189.63.204	Undefiend
145	51.6.67.90	GB
146	174.42.140.109	US
147	246.15

In [6]:
for i, item in enumerate(df.action.unique()):
    if item.count('/') == 2:
        # заносим в Products
        print(item.split('/'), i)
        product = item.split('/')
        try:
            db.request_insert_two('Products', 'category, name', product[0], product[1])
        except:
            print('Product already in database\n')

['canned_food', 'sprats', ''] 4
['canned_food', 'pate_of_tuna', ''] 5
['caviar', 'black_caviar', ''] 7
['frozen_fish', 'pike', ''] 9
['frozen_fish', 'shark', ''] 10
['fresh_fish', 'salmon', ''] 14
['canned_food', 'midii', ''] 15
['semi_manufactures', 'crab_meat', ''] 16
['frozen_fish', 'peljad', ''] 17
['fresh_fish', 'codfish', ''] 19
['fresh_fish', 'crucian', ''] 20
['semi_manufactures', 'stuffing_eel', ''] 21
['semi_manufactures', 'soup_set', ''] 22
['fresh_fish', 'tuna', ''] 24
['fresh_fish', 'marlene', ''] 26
['fresh_fish', 'herring', ''] 27
['semi_manufactures', 'squid_rings', ''] 30
['caviar', 'squash_caviar', ''] 31
['frozen_fish', 'salmon', ''] 34
['caviar', 'red_caviar', ''] 37
['fresh_fish', 'smelt', ''] 38
['semi_manufactures', 'salmon_cutlet', ''] 48
['frozen_fish', 'eel', ''] 58
['frozen_fish', 'carp', ''] 72


In [7]:
for i, item in enumerate(df.action):
    if 'pay?' in item:
        # zanosim v Transactions
        print('try oplat', item, i)
        u_id = db.request_select('id', 'Users', 'ip', df.ip[i])
        db.request_insert_two('Transactions', 'dtime, u_id', df.date[i], u_id[0][0])
    elif 'pay_' in item:
        # upd Transctions with same id pole finished to True
        print('opla4 korz uspechno', item, i)
        u_id = db.request_select('id', 'Users', 'ip', df.ip[i])
        print(u_id)
        t_id = db.request_select_one('id', 'Transactions', 'u_id', u_id[0][0], 'dtime')
        print(t_id)
        db.request_update('Transactions', 'finished', True, 'id', t_id[0][0])
    elif 'cart' in item:
        # po tz nikak v bd ne otslezhivaetsya
        print('sost korz', item, i)
    # zanosim v Actions
    print('parsing Actions', item, i)
    u_id = db.request_select('id', 'Users', 'ip', df.ip[i])
    db.request_insert_three('Actions', 'path, dtime, u_id', df.action[i], df.date, u_id[0][0])

sh/ 4941
parsing Actions index 4942
parsing Actions caviar/ 4943
parsing Actions frozen_fish/ 4944
parsing Actions index 4945
parsing Actions caviar/ 4946
try oplat pay?user_id=89893922693&cart_id=8881 4947
parsing Actions pay?user_id=89893922693&cart_id=8881 4947
opla4 korz uspechno success_pay_8881/ 4948
[(226,)]
[(154,)]
parsing Actions success_pay_8881/ 4948
parsing Actions canned_food/pate_of_tuna/ 4949
parsing Actions canned_food/ 4950
parsing Actions canned_food/midii/ 4951
try oplat pay?user_id=80802012147&cart_id=8877 4952
parsing Actions pay?user_id=80802012147&cart_id=8877 4952
opla4 korz uspechno success_pay_8877/ 4953
[(227,)]
[(155,)]
parsing Actions success_pay_8877/ 4953
parsing Actions index 4954
parsing Actions index 4955
parsing Actions fresh_fish/ 4956
parsing Actions index 4957
parsing Actions index 4958
parsing Actions caviar/ 4959
parsing Actions caviar/black_caviar/ 4960
parsing Actions frozen_fish/ 4961
parsing Actions frozen_fish/shark/ 4962
parsing Actions fr

In [5]:
conn = sqlite3.connect("database")
cursor = conn.cursor()
cursor.execute('''SELECT * FROM Transactions as A WHERE A.u_id = '[(178,)]' ORDER BY A.dtime DESC LIMIT 1''')
data = cursor.fetchall()
conn.close()
print(data)

[]
